In [1]:
#default_exp liftover

In [1]:
#hide
%load_ext autoreload
%autoreload 2

# Liftover module

> Liftover genodata and sumstat

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import gzip
from liftover import get_lifter

In [3]:
#export
class Liftover:
    def __init__(self,fr='hg19',to='hg38'):
        self.fr,self.to = fr,to
        self.chainmap = get_lifter(fr, to)
        #self.fasta 
              
    def variants_liftover(self,chrom,pos):
        if len(chrom) == 1:
            chrom = [chrom]*len(pos)
        lchr,lpos = [],[]
        for c,p in zip(chrom,pos):
            new_c,new_p = self.chrpos_liftover(c,p)
            lchr.append(new_c)
            lpos.append(new_p)
        return lchr,lpos
    
    def chrpos_liftover(self,chrom,pos):
        try:
            if str(chrom) in ['X','chrX','23']:
                new_c,new_p,_ = self.chainmap['X'][pos][0]
                return 23,new_p
            elif str(chrom) in ['Y','chrY','24']:
                new_c,new_p,_ = self.chainmap['Y'][pos][0]
                return 24,new_p
            elif str(chrom) in ['M','chrM','25','MT']:
                new_c,new_p,_ = self.chainmap['M'][pos][0]
                return 25,new_p
            else:
                new_c,new_p,_ = self.chainmap[int(chrom)][pos][0]
            return int(new_c[3:]),new_p
        except:
            return 0,0
           
        #The function to liftover bim
    def bim_liftover(self,bim):
        new_bim = bim.copy()
        lchr,lpos = self.variants_liftover(bim.chrom,bim.pos)
        new_bim.chrom =lchr
        new_bim.pos = lpos
        new_bim.snp = 'chr'+new_bim[['chrom','pos','a0','a1']].astype(str).agg(':'.join, axis=1)
        return new_bim

    
    def sumstat_liftover(self,ss,rename=True):
        new_ss = ss.copy()
        lchr,lpos = self.variants_liftover(ss.CHR,ss.POS)
        new_ss.CHR =lchr
        new_ss.POS = lpos
        if rename:
            new_ss.SNP = 'chr'+new_ss[['CHR','POS','A0','A1']].astype(str).agg(':'.join, axis=1)
        return new_ss
    
    def vcf_liftover(self,vcf,vcf_out=None,remove_missing = True):
        if vcf_out is None:
            vcf_out = vcf[:-7]+'_'+self.fr+'To'+self.to+vcf[-7:]
        count_fail,total= 0,0
        with gzip.open(vcf, 'rt') as ifile:
            with gzip.open(vcf_out,'wt') as ofile:
                for line in ifile:
                    if line.startswith("#"):
                        ofile.write(line)
                    else:
                        variant = [x for x in line.split('\t')]
                        new_c,new_p = self.chrpos_liftover(variant[0],int(variant[1]))
                        total +=1
                        if new_c == 0:
                            count_fail +=1
                            if remove_missing:
                                continue
                        variant[0] = str(new_c)
                        variant[1] = str(new_p)
                        variant[2] = 'chr'+':'.join(variant[:2]+variant[3:5])
                        ofile.write('\t'.join(variant))
            ofile.close()       
        ifile.close()
        if remove_missing:
            print("Total number SNPs ",total,". Removing SNPs failed to liftover ", count_fail)
        else:
            print("Total number SNPs ",total,". The number of SNPs failed to liftover ", count_fail,". Their chr and pos is replaced with 0, 0")


        
    def region_liftover(self,region):
        imp_cs,imp_start = self.chrpos_liftover(region[0],region[1])
        imp_ce,imp_end = self.chrpos_liftover(region[0],region[2])
        if imp_cs !=imp_ce:
            raise ValueError('After liftover, the region is not in the same chromosome anymore.')
        return imp_cs,imp_start,imp_end
    
    def df_liftover(self):
        pass


In [107]:
geno.bim.shape[0]

424

In [137]:
from pathlib import Path
from cugg.genodata import *
from cugg.sumstat import Sumstat
from cugg.liftover import Liftover
def main(input_path,output_path,fr='hg19',to='hg38',remove_missing=True,rename=True):
    lf = Liftover(fr,to)
    print("liftover from " + fr +"to" +to)
    print("Removing SNPs failed to liftover is", remove_missing)
    #file type detection, sumstats, plink, vcf,gvcf, >>>future bgen
    input_path = Path(input_path)
    input_suffixes = set(input_path.suffixes)
    output_path = Path(output_path)
    if not input_path.exists(): print("The file is not exist:", input_path)
    if input_path.suffix in ['.bim','.bed','.fam']:
        geno = Genodata(str(input_path.with_suffix('.bed')))
        new_bim = lf.bim_liftover(geno.bim)
        idx = new_bim.chrom == 0
        if remove_missing:
            geno.bim = new_bim
            geno.extractbyidx(~idx)
            geno.export_plink(output_path.with_suffix('.bed'))
            print("Total number SNPs ",new_bim.shape[0],". Removing SNPs failed to liftover ", sum(idx))
        else:
            write_bim(output_path.with_suffix('.bim'),new_bim)
            print("Total number SNPs ",new_bim.shape[0],". The number of SNPs failed to liftover ", sum(idx),". Their chr and pos is replaced with 0, 0")
    elif len(input_suffixes.intersection(['.gvcf','.vcf']))>0:
        lf.vcf_liftover(input_path,output_path,remove_missing)
    else:
        print("This file is considered as sumstat format file")
        sums = Sumstat(input_path,rename=rename)
        new_sums = lf.sumstat_liftover(sums.ss,rename)
        idx = new_sums.CHR == 0
        if remove_missing:
            new_sums[~idx].to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)
            print("Total number SNPs ",new_sums.shape[0],". Removing SNPs failed to liftover ", sum(idx))
        else:
            new_sums.to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)
            print("Total number SNPs ",new_sums.shape[0],". The number of SNPs failed to liftover ", sum(idx),". Their chr and pos is replaced with 0, 0")

In [93]:
from glob import glob

In [96]:
from pathlib import Path

In [108]:
tmp = Path('data/GH.AR.SAD.P1.001.0_X3547_S42_1180478_GVCF.hard-filtered.gvcf.gz')

In [110]:
tmp.suffix[1:] in ['bim','bed','fam']

False

In [106]:
tmp.with_suffix('.bed')

Path('test.bed')

### 1.Test liftover vcf and gvcf

In [6]:
lf = Liftover('hg19','hg38')

In [ ]:
lf.chainmap[22][50549067]

In [131]:
vcf ='data/GH.AR.SAD.P1.001.0_X3547_S42_1180478_GVCF.hard-filtered.gvcf.gz'

In [135]:
main(vcf,'data/new_hg19_hg38_test.gvcf.gz',remove_missing=True)

liftover from hg19tohg38
Removing SNPs failed to liftover is True
Total number SNPs  93816694 . The number of SNPs failed to liftover  59995


In [56]:
lf.region_liftover([5,272741,1213528-900000])

(5, 272626, 313413)

In [ ]:
lf.vcf_liftover(vcf)

In [23]:
59995/93816694

0.0006394917305442463

### 2.Test liftover plink

In [ ]:
1

In [1]:
from cugg.genodata import Genodata

In [3]:
geno = Genodata('/mnt/mfs/statgen/alzheimers-family/linkage_files/geno/full_sample/bfiles/full_sample.bed')

In [33]:
geno = Genodata('/mnt/mfs/statgen/guangyou/imputation/genome/othergenes/UKB_exome_othergenes.bed')

In [4]:
main('/mnt/mfs/statgen/guangyou/imputation/genome/othergenes/UKB_exome_othergenes.bed','test2.bed')

In [ ]:
main('MWE_region_extraction/100521_UKBB_Hearing_aid_f3393_expandedwhite_15601cases_237318ctrl_500k_PC1_PC2_f3393.regenie.snp_stats.gz','test_sumstats.sumstats.gz')

liftover from hg19tohg38
Removing SNPs failed to liftover is True
This file is considered as sumstat format file


In [34]:
geno

bim:      chrom                  snp   cm        pos a0 a1     i
0         1    chr1:55039741:G:C  0.0   55039741  G  C     0
1         1    chr1:55039742:G:A  0.0   55039742  G  A     1
2         1    chr1:55039749:G:C  0.0   55039749  G  C     2
3         1    chr1:55039750:T:C  0.0   55039750  T  C     3
4         1    chr1:55039753:T:C  0.0   55039753  T  C     4
...     ...                  ...  ...        ... .. ..   ...
1408     11  chr11:116832956:T:C  0.0  116832956  T  C  1408
1409     11  chr11:116832976:C:G  0.0  116832976  C  G  1409
1410     11  chr11:116832977:T:G  0.0  116832977  T  G  1410
1411     11  chr11:116832978:T:G  0.0  116832978  T  G  1411
1412     11  chr11:116832980:T:C  0.0  116832980  T  C  1412

[1413 rows x 7 columns] 
 fam:            fid      iid father mother gender trait       i
0       1000019  1000019      0      0      2    -9       0
1       1000078  1000078      0      0      2    -9       1
2       1000081  1000081      0      0      1    -9  

In [90]:
geno.extractbyidx(~(geno.bim.chrom == 1))

In [92]:
geno.export_plink('test1.bed')


Writing BED: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]

Writing FAM... 

done.
Writing BIM... done.


### Test liftover sumstat

In [4]:
from cugg.sumstat import Sumstat
from cugg.liftover import Liftover
def gwas_liftover(input_path,output_path,output_unmapped,output_mapped,fr='hg19',to='hg38',remove_missing=False):
    lf = Liftover(fr,to)
    print("reading GWAS sumstat")
    sums = Sumstat(input_path)
    print("liftover from" + fr +"to" +to)
    sums1 = lf.sumstat_liftover(sums.ss)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_unmapped, compression='gzip', sep = "\t", header = True, index = False)
        sums1[sums1.CHR != 0].to_csv(output_mapped, compression='gzip', sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)

In [ ]:
def gwas_liftover(input_file,output_path=None,fr='hg19',to='hg38',remove_missing=False):
    if output_path is None:
        output_path = os.path.dirname(input_file)+'/'
    basename = os.path.basename(input_file)
    lf = Liftover('hg19','hg38')
    print("reading GWAS sumstat")
    sums = Sumstat(input_path)
    print("liftover from" + fr +"to" +to)
    sums1 = lf.sumstat_liftover(sums.ss)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_unmapped, compression='gzip', sep = "\t", header = True, index = False)
        sums1[sums1.CHR != 0].to_csv(output_mapped, compression='gzip', sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)

In [ ]:
sumstats_lifted = f'{cwd}/{_input:bnn}.hg38.sumstats.gz',
sumstats_unmapped = f'{cwd}/{_input:bnn}.hg38.sumstats_unmapped.gz',
sumstats_mapped = f'{cwd}/{_input:bnn}.hg38.sumstats_mapped.gz'

In [37]:
import os

In [41]:
tmp = os.path.basename(input_path)

In [42]:
os.path.splitext(tmp)

('100521_UKBB_Combined_f2247_f2257_expandedwhite_93258cases_237318ctrl_500k_PC1_PC2_f2247_f2257.regenie.snp_stats',
 '.gz')

In [40]:
os.path.dirname(input_path)+'/'

'/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/2021_10_07_combined_500K/'

In [19]:
sums = Sumstat(input_path)

In [33]:
lf = Liftover('hg19','hg38')

In [34]:
lf.sumstat_liftover(sums.ss[:10])

,CHR,POS,REF,ALT,SNP,BETA,SE,P
0,1,13259,G,A,chr1:13259:G:A,0.434586,0.175780,0.014801
1,1,17569,C,A,chr1:17569:C:A,-0.030568,0.795968,0.969366
2,1,17641,G,A,chr1:17641:G:A,-0.078881,0.108663,0.467883
3,1,30741,C,A,chr1:30741:C:A,-1.599610,0.990472,0.044798
4,1,57222,T,C,chr1:57222:T:C,0.031666,0.121422,0.794253
5,1,58396,T,C,chr1:58396:T:C,0.366266,0.172004,0.035663
6,1,62157,G,A,chr1:62157:G:A,-0.147251,0.296105,0.618983
7,1,62595,C,T,chr1:62595:C:T,0.356993,0.171623,0.040096
8,1,69487,G,A,chr1:69487:G:A,-0.559373,0.853882,0.512407
9,1,69569,T,C,chr1:69569:T:C,0.232690,0.216585,0.282662


In [25]:

def main(input_path,output_path,remove_missing):
    sums = read_regenie(input_path)
    sums1 = sumstat_liftover(sums)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_path, sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, sep = "\t", header = True, index = False)

In [18]:
input_path = '/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/2021_10_07_combined_500K/100521_UKBB_Combined_f2247_f2257_expandedwhite_93258cases_237318ctrl_500k_PC1_PC2_f2247_f2257.regenie.snp_stats.gz'
output_path = ''
remove_missing = True

In [2]:
main(input_path,output_path,remove_missing)

In [44]:
':'.join([1,'1'])

TypeError: sequence item 0: expected str instance, int found

In [ ]:
gen

## test chr22

In [5]:
region = [22,50519304,50549676]
exome_sumstats = Sumstat('/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/010522_f2247_hearing_diff_200K_imputed/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl_PC1_2_f2247.regenie.snp_stats.gz')
exome_sumstats.extractbyregion(region)

this region [22, 50519304, 50549676] has 669 SNPs in Sumstat


In [ ]:
print('1.2. LiftOver the region')
hg38toimpref = Liftover('hg38','hg19')
imp_region = hg38toimpref.region_liftover(region)
imput_sumstats.extractbyregion(imp_region)

print('1.3. Regional SNPs Liftover')
impreftohg38 = Liftover(imp_ref,'hg38') #oppsite with hg38toimpref
impreftohg38.